# IRS Project Report
## I. Introduction

This report will be a brief presentation on the pip-installable package of IRS, Infra-Red Simulator. Infrared (IR) spectroscopy is an analytical technique used to identify and study chemical compounds based on how they absorb infrared radiation. The package visualizes a given molecule, and performs calculations in order to simualte the infrared spectrum. The main results of the functions, along with the project challenges limitations will be demonstrated below. 
Before going through the code, a refresher on relevant chemistry notions, vibrational spectroscopy, is discussed.

##  II. Relevant Chemistry
When molecules are exposed to IR light, specific vibrational modes of their chemical bonds are excited, depending on the frequency of the radiation. For a vibrational transition to be IR-active, it must involve a change in the dipole moment of the molecule.

###  Vibrational Theory

Under the harmonic approximation, atoms in a molecule are modeled as point masses connected by springs, representing chemical bonds. These atoms undergo vibrational motions (stretching, bending, etc.) around their equilibrium positions. The vibrational frequencies $\nu$ are related to the second derivatives of the molecular energy with respect to nuclear displacements — these derivatives form the Hessian matrix.

In a simple diatomic molecule, the vibrational frequency can be approximated using the harmonic oscillator model:
$$
\nu = \frac{1}{2\pi c} \sqrt{\frac{k}{\mu}}
$$

Where:

- $\nu$ is the vibrational frequency (in Hz),`
- $c$ is the speed of light (in cm/s),
- $k$ is the force constant of the bond (in N/m),
- $\mu$ is the reduced mass of the two atoms, given by:

$$
\mu = \frac{m_1 m_2}{m_1 + m_2}
$$

This model explains how heavier atoms (larger $\mu$) or weaker bonds (smaller $k$) lead to lower vibrational frequencies. Conversely, light atoms connected by stiff bonds vibrate at higher frequencies.

In reality, molecules consist of many atoms and therefore exhibit multiple vibrational modes: $3N - 6$ for nonlinear molecules, and $3N - 5$ for linear molecules, where $N$ is the number of atoms. Each of these normal modes corresponds to a specific pattern of atomic motion and can be analyzed for infrared (IR) activity.

A mode is IR-active if it involves a change in the dipole moment during the vibration. These active modes appear as peaks in the IR spectrum. The frequency and intensity of each peak provide insight into the bond types and functional groups present in the molecule.

Each functional group (such as –OH, –NH₂, C=O, etc.) has characteristic absorption bands in the IR region, typically between 4000 and 400 cm⁻¹. This results in a unique "molecular fingerprint" for every compound. The IR spectrum plots frequency (in wavenumbers, cm⁻¹) on the x-axis and intensity or transmittance on the y-axis, reflecting how much IR radiation is absorbed at each frequency.

Infrared spectroscopy is a powerful analytical technique because it allows chemists to identify the presence of specific functional groups within a molecule by examining their characteristic absorption bands. It provides a rapid and non-destructive method for confirming molecular structure. Additionally, it is commonly employed to monitor the progress of chemical reactions by observing changes in the IR spectrum that correspond to bond formation or cleavage. Because each molecule has a distinct IR absorption pattern, the technique is also valuable for distinguishing between similar compounds or assessing sample purity.

## III. Motivation
The initial project idea was to create a molecule combiner/analyzer but was quickly dismissed due to the generality and the simplicity of the final result. Sharing an interest in quantum chemistry, our team decided to create a tool combining quantum mechanical simulation and utility for chemists, which led to the idea of a computational Infra-Red Simulator. Combining vibrational spectroscopy and computational chemistry, we aimed to bridge theoretical methods with practical visualization tools. The project evolved into an Infra-Red Simulator capable of predicting IR spectra from molecular structures using quantum mechanical principles. Our goal is to make the vibrational analysis of molecules more accessible, allowing the user to visualize molecular vibrations and functional group behavior through a simple interface.
In addition to the quantum chemistry-based simulations, we recognized the need for a faster, approximate method to estimate IR spectra based on functional groups. This approach is particularly useful since it gives an empirical estimate of the IR spectrum, grounded in experimentally observed characteristic peaks, rather than purely theoretical calculations.
Using RDKit, QM packages (Psi4, ORCA), and Streamlit, this package enables chemists to generate and explore IR spectra from molecular structures in an intuitive environment.

## IV. Usage Example
After installing the package (as described in the README), users gain access to the full suite of functions it offers. This project is modular in design and can be divided into three independent components, each of which can be run separately: two quantum mechanical modules (using ORCA and Psi4, respectively) and a structural analysis module focused on identifying key features within a molecule.

The functions associated with each of these components are outlined and explained below.

### QM approach: Psi4 and ORCA

The combined streamlit interface features a first part of molecular visualisation, then the quantum calculatiions are performed differently using Psi4 and ORCA.


#### Molecular Visualization
`name_to_smiles(name)`

This function converts a name to SMILES using PubChems API.

In [ ]:
import irspectrum-sim
output = irspectrum-sim.name_to_smiles("ethanol")
print(output)


`generate_3d_molecule(smiles)`

This function converts a SMILES string into an 3D molecule.It begins by parsing the SMILES into an RDKit molecule object, then adds explicit hydrogens to ensure correct valency. To generate 3D coordinates, it applies the ETKDG algorithm with a fixed random seed (42) for reproducibility; if the initial embedding fails, a fallback attempt is made. The resulting structure is then optimized using the Universal Force Field (UFF), allowing up to 200 iterations. 

In [ ]:
import irspectrum-sim
from rdkit import Chem
from rdkit.Chem import AllChem
output = irspectrum-sim.generate_3d_molecule("CCO")
print(output)


`mol_to_3dviewer(mol)`

The molecule is then visualized in 3D with the above function. This function visualizes a 3D molecule by first converting the RDKit molecule into a mol block format, a textual representation of its structure. It then initializes a py3Dmol viewer with defined dimensions and loads the molecular model into the viewer. The view of the input molecule is automatically zoomed to fit the entire molecule for optimal display. The result is shown in the interface with the function `show_3dmol(viewer)`.

In [ ]:
#mol_to_3dviewer(mol)
import irspectrum-sim
from rdkit import Chem
import py3Dmol
mol = Chem.MolFromSmiles("CCO")
mol = Chem.AddHs(mol)
output_1 = irspectrum-sim.mol_to_3dviewer(mol)
print(output_1)

#show_3dmol(viewer)
import irspectrum-sim
output_2 = irspectrum-sim.show_3dmol(output_1)
print(output_2)

#### Plotting the IR Spectrum

`plot_ir_spectrum(freqs, intensities, sigma=20, scale_factor=0.97)`
This function generates a simulated IR spectrum by scaling the input frequencies, constructing a smooth wavenumber range, and adding Gaussian peaks for each vibrational mode. It converts the absorbance data to percent transmittance, formats the plot with standard IR conventions (including axis inversion), and returns the resulting figure. The `freqs` and `intensities` variables used in the plot are obtained from quantum mechanical calculations using either Psi4 or ORCA.

### Quantum Mechanical Calculations 

#### Psi4

First of all, a caching mechanism is implemented with the `cached_geometry_optimization(smiles, method)` function and the Streamlit `@st.cache_resource` decorator. This ensures that if the same SMILES string and computational method are provided again, the geometry optimization is not recomputed. Instead, the previously stored result is reused, significantly improving performance.
`smiles_to_optimized_geometry(smiles, method)`

This function prepares and optimizes a molecular geometry using quantum chemistry. It first generates an initial 3D structure with RDKit, then converts it into the format required by Psi4 by extracting atomic coordinates and reformatting them as "element x y z" entries. The function returns both the Psi4 and RDKit molecule objects. This optimization step refines the initial structure by accounting for electronic effects beyond the reach of classical force fields.

`psi4_calculate_frequencies(molecule, selected_method)`

This function calculates the vibrational frequencies from normal modes using the Psi4 package. It starts by allocating memory and configuring output settings, then runs the `frequency` calculation with the selected method, recording the execution time. Vibrational frequencies are extracted from the wavefunction object and converted to a NumPy array. <br>

The function attempts to retrieve IR intensities from multiple possible keys (to ensure compatibility across Psi4 versions), validating the data and falling back on dummy values if needed. It returns the frequencies, intensities, runtime, and a flag indicating whether real IR data was available. This step is key for generating IR spectra and confirming that the molecule is at a true energy minimum. The method used (e.g., HF, B3LYP, MP2) defines the level of theory, trading off between speed and accuracy. This gives three version of infra-red spectrums of the molecule, which can be compared to an experimental spectrum for accuracy.

The `method` parameter in these functions refers to the quantum chemistry method used for calculations. Common options include:

- **HF**: Hartree-Fock (fastest but least accurate)
- **B3LYP**: Popular density functional theory (DFT) method (good balance of accuracy and speed)
- **MP2**: Second-order Møller–Plesset perturbation theory (more accurate but slower)

The method is typically combined with a basis set, such as "B3LYP/6-31G*" or "HF/3-21G".

`guess_charge_multiplicity(mol)`

This function calculates two essential parameters for quantum calculations: the formal charge and spin multiplicity. The formal charge is retrieved from the RDKit molecule, while the multiplicity is estimated based on the number of unpaired electrons. If none are found, the default multiplicity is 1.

In [ ]:
import irspectrum-sim
from rdkit import Chem
mol = Chem.MolFromSmiles("CCO")
output = irspectrum-sim.guess_charge_multiplicity(mol)
print(output)

### ORCA

`write_orca_input(mol, output_dir, base_name, method, charge, multiplicity)`

This function prepares the input file for ORCA. It extracts atomic coordinates from the RDKit molecule and writes them in XYZ format, along with charge, multiplicity, and the selected computational method. The `method` parameter in this function refers to the quantum chemistry method used for calculations, as with Psi4. The following options are possible with the ORCA software:

- **PBE0**: Density functional theory base method (faster but similar to B3LYP)
- **B3LYP**: Popular density functional theory method (good balance, intermediate method)
- **wB97X-D3**: Range-separated hybrid DFT method with dispersion correction included (accurate but also the slowest)

These methods are combined with the "def2-SVP" basis set in the case of the PBE0 and B3LYP methods, and with the "def2-TZVP" (TZ stands for triple zeta) in the case of the MP2 method. T

`run_orca`, `parse_orca_output`

These The run_orca function executes the ORCA calculation by calling the ORCA binary on the generated input file and directing the output to a .out file. Once the calculation is complete, the parse_orca_output function processes this output file to extract vibrational frequencies and IR intensities from the "IR SPECTRUM" section. The results are returned as NumPy arrays, ready for spectrum plotting. If the output is missing or contains no valid data, appropriate error messages are shown to inform the user.

The intermidate files generated by ORCA are then removes by the`cleanup_orca_files` function.

`handle_psi4_calculation(smiles, selected_method, freq_scale, peak_width, debug_mode)`

This function performs a vibrational frequency calculation using Psi4 for a given molecule and method. It pulls out the vibrational frequencies and the strength of their IR signals from the results. If the intensity data is missing or doesn’t look right, the function handles that gracefully by using placeholder values. In the end, it returns the frequencies, intensities, how long the calculation took, and whether the intensity data is trustworthy.

`handle_orca_calculation(smiles, selected_method, orca_path, output_dir, freq_scale, peak_width, debug_mode)`

This function handles the full process of generating an IR spectrum using ORCA. It starts by turning the input SMILES into a 3D structure, then prepares everything ORCA needs to run, like setting up a file that tells ORCA to optimize the molecule’s shape and calculate its vibrations. After kicking off the ORCA job, it waits for the results, then digs through the output to pull out the important info: the vibrational frequencies and how intense each one is. Finally, it tweaks the data a bit (like scaling the frequencies and smoothing the peaks) and plots the IR spectrum for you to see.

The debug_mode in both these functions is a parameter used to control whether intermediate files and detailed calculation data are retained or cleaned up after the simulation runs. When debug_mode is set to True, these files are kept to help with troubleshooting and debugging, allowing developers or users to inspect the calculation process in more detail. When set to False, the functions perform cleanup to remove temporary files, keeping the workspace tidy for regular use. 
`handle_functional_groups_calculation(smiles)`

This function gives you a quick IR spectrum estimate based on the molecule’s functional groups. It takes a SMILES string, turns it into a molecule, and checks which common groups are present, like alcohols, ketones, etc. Then, it uses that info to sketch out an approximate IR spectrum by calling another function that handles the details (like assigning peaks and drawing the graph). It’s not as precise as full quantum chemistry, but it’s much faster and great for getting a rough idea of what to expect.

Finally, we have `handle_ir_calculation`.

This function acts like a traffic director for your IR spectrum calculations. Based on what the user picks—whether it’s Psi4, ORCA, or a quicker estimate using functional groups, it sends the molecule to the right tool for the job. It keeps things organized behind the scenes and makes sure the app runs the correct process without the user having to worry about the details.

## Structural approach

`get_functional_groups`

This function identifies and counts the presence of specific functional groups in a molecule based on its SMILES representation. For this, it uses a predefined dictionary of SMARTS (`FUNCTIONAL_GROUPS`) patterns, where each SMARTS describes the chemical structure of a functional group. For each SMARTS, the function searches for corresponding substructures in the molecule using RDKit's substructure matching. The output is a dictionnary where the keys are names of the functional groups, and the values indicate how many times each molecule appears in the molecule.

In [ ]:
import irspectrum-sim
from rdkit import Chem
from collections import defaultdict
output = irspectrum-sim.get_functional_groups("C=CC(=O)O")
print(output)


 `detect_main_functional_groups`
 
The function refines the output of the `get_functional_groups` by removing overlaps and avoiding double-counting. It applies correction rules to prioritize the main functional groups and subtract counts for the simpler groups they contain. The output is a dictionnary where the keys are the main functional groups, and the values indicate how many times each molecule appears in the molecule.

In [ ]:
import irspectrum-sim
output = irspectrum-sim.detect_main_functional_groups("C=CC(=O)O")
print(output)


 `count_ch_bonds`

This function takes as an input a SMILES of a molecule and as an output how many hydrogen atoms are bonded to carbon atoms of different hybridization (sp^3, sp^2, sp) in the molecule.

In [ ]:
import irspectrum-sim
from rdkit import Chem
output=irspectrum-sim.count_ch_bonds("C=CC(=O)O")
print(output)


 `count_carbon_bonds`

This function takes as an input the SMILES of a molecule and returns how many C-C, C=C, and C≡C bonds there are in this molecule.

In [ ]:
import irspectrum-sim
from rdkit import Chem
output = irspectrum-sim.count_carbon_bonds("C=CC(=O)O")
print(output)


Finally, `analyze_molecule`combines the three functions mentionned above to generate a dictionnary with all keys funtional groups, C-H bonds and C-C bonds.

## V. Difficulties

### Quantum Mechanical Calculations 

Despite the power and theoretical accuracy of quantum mechanical methods like Hartree–Fock (HF), density functional theory (DFT) with B3LYP, and Møller–Plesset perturbation theory (MP2), several limitations affect their use in practical infrared (IR) spectrum simulation. First, these calculations are computationally demanding: geometry optimizations and frequency evaluations scale poorly with molecular size, making them impractical for large molecules without significant computational resources or method-specific optimizations. The accuracy of the results also heavily depends on the choice of method and basis set. For example, HF tends to underestimate electron correlation, while MP2, although more accurate, is computationally expensive and can overcorrect certain features. B3LYP offers a good compromise but is still sensitive to basis set selection.

### Structural Approach

Regarding the structural approach method, there are several limitations. First, it reapresents all peaks as symmetric Gaussian functions, this fails to capture the asymmmetric or broadened peaks. The fixed frequencies and intensities in the disctiionary.py (`FUNCTIONAL_GROUPS_IR`) ignore environnmental effects, such as conjugation, ring strain...etc, which cause peak shifts. The linear addition of peaks also overlooks vibrational coupling and combination bands, leading to inaccrate intensities in the fingerprint zone (<1500cm⁻¹). 
In additon, the dictionnary in get_functional_groups (`FUNCTIONAL_GROUPS`) lacks the functional groups derived from sulfur, boron and phosphorus.
The code's reliance on SMILES also introduces the limitation that IR sepctra depend on 3D molecular geometry, but SMILES lacks certain conformational informations.
Lastly, the `validate_smiles` function imposes additional constraints that further narrow the scope of molecules the code can handle. The function:
- Accepts only molecules composed of the atoms: C, H, N, O, I, F, Cl and Br.
- Rejects any molecule with charged atoms.
- Imposes a rule on aromatic rings containing N or O, requiring that they contain exactly 5 or 6 carbons.

## VI. Output Example 

An output example is displayed below, using the ethanol molecule: <br>

<div style="text-align:center;">

For the functional group approach, the output is displayed below:

<img src="../assets/notebook-image-1.png" alt="Functional group method IR spectrum" width="1400"/>

<p>For the Psi4 method, the output of the HF/STO-3G is displayed below:</p>

<img src="../assets/notebook-image-2.png" alt="Psi4 method IR spectrum" width="1400"/>

<p>For the ORCA method, the output of the B3LYP method is displayed below:</p>

<img src="../assets/notebook-image-3.png" alt="ORCA method IR spectrum" width="1400"/>

</div>

The slight differences in peaks represent the difference between a fully quantum mechanical method(Psi4), and a semi-empirical method(ORCA).

## VII Data Sources

The following sources were used to generate the `dict_fg_IR_data.json` dictionary:

- [NC State Organic Chemistry Textbook](https://ncstate.pressbooks.pub/organicchem/chapter/infrared-spectra-of-some-common-functional-groups/)  
  *Accessed May 23, 2025.*

- [IR Spectrum Table on StudyLib](https://studylib.net/doc/25856154/ir-spectrum-table)  
  *Accessed May 23, 2025.*

- [LibreTexts Chemistry](https://chem.libretexts.org/Courses/SUNY_Oneonta/Chem_322_Lecture_Content/05%3A_Organohalogen_and_Organometallic_Compounds/5.03%3A_Spectroscopic_Properties)  
  *Accessed May 23, 2025.*

- [SDBS (Spectral Database for Organic Compounds)](https://sdbs.db.aist.go.jp/SearchInformation.aspx), maintained by AIST Japan  
  *Accessed May 23, 2025.*
